<a href="https://colab.research.google.com/github/AaronDTY/weatherwise-aaron-teck/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [3]:
# Install required packages for Google Colab environment
# This cell will install all necessary packages

!pip install requests matplotlib pyinputplus


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=be9fd7bd6f36b3cf1659bedb4dfd30ece19fe9a8ccedf834c001b7c718371d99
  Stored in directory: /root/.cache/pip/wheels/a3/6b/89/3bec14288af8e0729d088fb41a785e583d49e772f2529c7676
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16226 sha256=f45aed99092346d6fb0c4cd3d2c6c7ee84ea49b048617f6d303e50d97cf05438
  Stored in directory: /root/.cache/pip/wheels/fc/40/7a/4d3b2dc2e80b4b3bbda89eec94fcefd6c8bd1101cc7bb89554
  Created wheel for stdiomask: filename=stdiomask

In [4]:
# Import standard libraries
import os
import sys
import json
import time
import re
from datetime import datetime, timedelta

# Import third-party libraries
import requests
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pyinputplus as pyip

# Configure matplotlib for better visualizations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Display environment information
print(f"Python version: {sys.version}")
# Access the version from the main matplotlib module
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"Working directory: {os.getcwd()}")

Python version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
Matplotlib version: 3.10.0
Working directory: /content


## 📦 Setup and Configuration
Import required packages and setup environment.

In [5]:
# Configuration settings for the WeatherWise application

# Weather API configuration
WEATHER_API_BASE_URL = "https://wttr.in/"
WEATHER_API_FORMAT = "j1"  # JSON format
MAX_FORECAST_DAYS = 3      # Maximum number of forecast days supported

# Application settings
DEFAULT_LOCATION = "London"  # Default location if none specified
REQUEST_TIMEOUT = 10         # Timeout for API requests in seconds
CACHE_DURATION = 1800        # Cache duration in seconds (30 minutes)

# Visualization settings
TEMP_COLORS = {"high": "#FF5733", "low": "#33A1FF"}  # Colors for temperature visualization
PRECIP_COLOR = "#3498DB"                              # Color for precipitation visualization

# Create cache directory if it doesn't exist
if not os.path.exists('cache'):
    os.makedirs('cache')
    print("Created cache directory for storing temporary data")

print("WeatherWise configuration loaded successfully")

Created cache directory for storing temporary data
WeatherWise configuration loaded successfully


## 🌤️ Weather Data Functions

In [6]:
# @title Default title text
def get_weather_data(location, forecast_days=3):
    """
    Retrieve weather data for a specified location using the wttr.in API.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-3)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    # Input validation
    if not location or not isinstance(location, str):
        location = DEFAULT_LOCATION
        print(f"Invalid location. Using default: {DEFAULT_LOCATION}")

    # Ensure forecast_days is within valid range
    if not isinstance(forecast_days, int) or forecast_days < 1 or forecast_days > MAX_FORECAST_DAYS:
        forecast_days = MAX_FORECAST_DAYS
        print(f"Invalid forecast days. Using maximum: {MAX_FORECAST_DAYS}")

    # Check cache first
    cache_file = f"cache/{location.lower().replace(' ', '_')}.json"
    if os.path.exists(cache_file):
        with open(cache_file, 'r') as f:
            cache_data = json.load(f)
            # Check if cache is still valid
            if time.time() - cache_data.get('timestamp', 0) < CACHE_DURATION:
                print(f"Using cached data for {location}")
                return cache_data['data']

    # Prepare API request
    url = f"{WEATHER_API_BASE_URL}{location}?format={WEATHER_API_FORMAT}&lang=en"

    try:
        # Make API request with timeout
        response = requests.get(url, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()  # Raise exception for HTTP errors

        # Parse JSON response
        weather_data = response.json()

        # Process and structure the data
        processed_data = process_weather_data(weather_data, forecast_days)

        # Save to cache
        cache_data = {
            'timestamp': time.time(),
            'data': processed_data
        }
        with open(cache_file, 'w') as f:
            json.dump(cache_data, f)

        return processed_data

    except requests.exceptions.Timeout:
        print(f"Error: Request timed out after {REQUEST_TIMEOUT} seconds")
        return {"error": "Request timed out. Please try again later."}

    except requests.exceptions.ConnectionError:
        print("Error: Connection error. Please check your internet connection.")
        return {"error": "Connection error. Please check your internet connection."}

    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            print(f"Error: Location '{location}' not found.")
            return {"error": f"Location '{location}' not found. Please check the spelling or try a different location."}
        else:
            print(f"HTTP Error: {e}")
            return {"error": f"HTTP Error: {e}"}

    except json.JSONDecodeError:
        print("Error: Invalid response from weather service.")
        return {"error": "Invalid response from weather service. Please try again later."}

    except Exception as e:
        print(f"Unexpected error: {e}")
        return {"error": f"Unexpected error: {e}"}

In [7]:
def process_weather_data(raw_data, forecast_days):
    """
    Process raw weather data from the API into a structured format.

    Args:
        raw_data (dict): Raw weather data from the API
        forecast_days (int): Number of days to include in the forecast

    Returns:
        dict: Processed weather data
    """
    # Initialize processed data structure
    processed_data = {
        "location": {},
        "current": {},
        "forecast": []
    }

    # Extract location information
    if "nearest_area" in raw_data and len(raw_data["nearest_area"]) > 0:
        area = raw_data["nearest_area"][0]
        processed_data["location"] = {
            "name": area.get("areaName", [{}])[0].get("value", "Unknown"),
            "country": area.get("country", [{}])[0].get("value", "Unknown"),
            "region": area.get("region", [{}])[0].get("value", "Unknown"),
            "latitude": area.get("latitude", "0"),
            "longitude": area.get("longitude", "0")
        }

    # Extract current weather conditions
    if "current_condition" in raw_data and len(raw_data["current_condition"]) > 0:
        current = raw_data["current_condition"][0]
        processed_data["current"] = {
            "temp_C": int(current.get("temp_C", 0)),
            "temp_F": int(current.get("temp_F", 0)),
            "weather_desc": current.get("weatherDesc", [{}])[0].get("value", "Unknown"),
            "humidity": int(current.get("humidity", 0)),
            "wind_speed_kmph": int(current.get("windspeedKmph", 0)),
            "wind_direction": current.get("winddir16Point", "Unknown"),
            "precipitation_mm": float(current.get("precipMM", 0)),
            "feels_like_C": int(current.get("FeelsLikeC", 0)),
            "visibility_km": int(current.get("visibility", 0)),
            "uv_index": int(current.get("uvIndex", 0)),
            "observation_time": current.get("observation_time", "Unknown")
        }

    # Extract forecast data
    if "weather" in raw_data:
        for i, day in enumerate(raw_data["weather"]):
            if i >= forecast_days:
                break

            date = day.get("date", "Unknown")
            astronomy = day.get("astronomy", [{}])[0]

            # Process hourly forecasts
            hourly_forecasts = []
            for hour in day.get("hourly", []):
                time_int = int(hour.get("time", 0))
                hour_of_day = time_int // 100

                hourly_forecasts.append({
                    "time": f"{hour_of_day:02d}:00",
                    "temp_C": int(hour.get("tempC", 0)),
                    "feels_like_C": int(hour.get("FeelsLikeC", 0)),
                    "weather_desc": hour.get("weatherDesc", [{}])[0].get("value", "Unknown"),
                    "precipitation_mm": float(hour.get("precipMM", 0)),
                    "chance_of_rain": hour.get("chanceofrain", "0"),
                    "humidity": int(hour.get("humidity", 0)),
                    "wind_speed_kmph": int(hour.get("windspeedKmph", 0)),
                    "wind_direction": hour.get("winddir16Point", "Unknown")
                })

            # Calculate daily min/max temperatures
            temps = [h["temp_C"] for h in hourly_forecasts]
            max_temp = max(temps) if temps else 0
            min_temp = min(temps) if temps else 0

            # Calculate total precipitation
            total_precip = sum(float(h["precipitation_mm"]) for h in hourly_forecasts)

            # Add day forecast to processed data
            processed_data["forecast"].append({
                "date": date,
                "max_temp_C": max_temp,
                "min_temp_C": min_temp,
                "total_precipitation_mm": round(total_precip, 1),
                "sunrise": astronomy.get("sunrise", "Unknown"),
                "sunset": astronomy.get("sunset", "Unknown"),
                "hourly": hourly_forecasts
            })

    return processed_data

## 📊 Visualisation Functions

In [8]:
def create_temperature_visualization(weather_data, output_type='display'):
    """
    Create visualization of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualization in the notebook
    """
    # Check for errors in weather data
    if "error" in weather_data:
        print(f"Error: {weather_data['error']}")
        return None

    # Check if forecast data exists
    if "forecast" not in weather_data or not weather_data["forecast"]:
        print("Error: No forecast data available for visualization")
        return None

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 6))

    # Prepare data for plotting
    dates = []
    max_temps = []
    min_temps = []

    # Current temperature for reference
    current_temp = weather_data["current"]["temp_C"]

    # Extract data from forecast
    for day in weather_data["forecast"]:
        date_obj = datetime.strptime(day["date"], "%Y-%m-%d")
        dates.append(date_obj)
        max_temps.append(day["max_temp_C"])
        min_temps.append(day["min_temp_C"])

    # Plot temperature data
    ax.plot(dates, max_temps, 'o-', color=TEMP_COLORS["high"], linewidth=2, markersize=8, label='Max Temperature')
    ax.plot(dates, min_temps, 'o-', color=TEMP_COLORS["low"], linewidth=2, markersize=8, label='Min Temperature')

    # Add current temperature marker
    ax.axhline(y=current_temp, color='green', linestyle='--', alpha=0.7, label='Current Temperature')

    # Fill between max and min temperatures
    ax.fill_between(dates, max_temps, min_temps, alpha=0.2, color='gray')

    # Add temperature labels
    for i, (max_t, min_t) in enumerate(zip(max_temps, min_temps)):
        ax.annotate(f"{max_t}°C", (dates[i], max_t), textcoords="offset points",
                    xytext=(0, 10), ha='center', fontweight='bold')
        ax.annotate(f"{min_t}°C", (dates[i], min_t), textcoords="offset points",
                    xytext=(0, -15), ha='center', fontweight='bold')

    # Configure axis and labels
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Temperature (°C)', fontsize=12)

    # Format x-axis dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a\n%b %d'))

    # Set y-axis range with some padding
    min_y = min(min_temps) - 3
    max_y = max(max_temps) + 3
    ax.set_ylim(min_y, max_y)

    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)

    # Add legend
    ax.legend(loc='best', frameon=True, facecolor='white', edgecolor='gray')

    # Add title
    location_name = weather_data["location"].get("name", "Unknown")
    country_name = weather_data["location"].get("country", "")
    plt.title(f'Temperature Forecast for {location_name}, {country_name}', fontsize=14, pad=20)

    # Add timestamp
    plt.figtext(0.02, 0.02, f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

    # Adjust layout
    plt.tight_layout()

    # Return or display based on output_type
    if output_type == 'figure':
        return fig
    else:
        plt.show()


In [9]:

def create_precipitation_visualization(weather_data, output_type='display'):
    """
    Create visualization of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualization in the notebook
    """
    # Check for errors in weather data
    if "error" in weather_data:
        print(f"Error: {weather_data['error']}")
        return None

    # Check if forecast data exists
    if "forecast" not in weather_data or not weather_data["forecast"]:
        print("Error: No forecast data available for visualization")
        return None

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 6))

    # Prepare data for plotting
    dates = []
    daily_precip = []
    hourly_times = []
    hourly_precip = []
    chance_of_rain = []

    # Extract data from forecast
    for day in weather_data["forecast"]:
        date_obj = datetime.strptime(day["date"], "%Y-%m-%d")
        dates.append(date_obj)
        daily_precip.append(day["total_precipitation_mm"])

        # Extract hourly data
        for hour in day["hourly"]:
            hour_time = datetime.strptime(f"{day['date']} {hour['time']}", "%Y-%m-%d %H:%M")
            hourly_times.append(hour_time)
            hourly_precip.append(float(hour["precipitation_mm"]))
            chance_of_rain.append(int(hour["chance_of_rain"]))

    # Plot daily precipitation as bars
    bar_width = 0.4
    bars = ax.bar(dates, daily_precip, width=bar_width, color=PRECIP_COLOR, alpha=0.7, label='Daily Precipitation')

    # Add precipitation labels
    for bar, precip in zip(bars, daily_precip):
        if precip > 0:
            height = bar.get_height()
            ax.annotate(f"{precip} mm", xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

    # Create secondary y-axis for chance of rain
    ax2 = ax.twinx()

    # Plot chance of rain as a line
    ax2.plot(hourly_times, chance_of_rain, 'r-', alpha=0.6, label='Chance of Rain (%)')

    # Configure primary axis (precipitation)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Precipitation (mm)', fontsize=12)

    # Configure secondary axis (chance of rain)
    ax2.set_ylabel('Chance of Rain (%)', fontsize=12, color='red')
    ax2.tick_params(axis='y', colors='red')
    ax2.set_ylim(0, 100)

    # Format x-axis dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%a\n%b %d'))

    # Set y-axis range with some padding
    max_precip = max(daily_precip) if daily_precip else 0
    ax.set_ylim(0, max_precip * 1.2 + 1)  # Add padding

    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)

    # Add legends for both axes
    lines1, labels1 = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines1 + lines2, labels1 + labels2, loc='upper left', frameon=True, facecolor='white', edgecolor='gray')

    # Add title
    location_name = weather_data["location"].get("name", "Unknown")
    country_name = weather_data["location"].get("country", "")
    plt.title(f'Precipitation Forecast for {location_name}, {country_name}', fontsize=14, pad=20)

    # Add timestamp
    plt.figtext(0.02, 0.02, f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

    # Adjust layout
    plt.tight_layout()

    # Return or display based on output_type
    if output_type == 'figure':
        return fig
    else:
        plt.show()

## 🤖 Natural Language Processing

In [10]:
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    # Convert question to lowercase for easier matching
    question = question.lower()

    # Initialize result dictionary
    result = {
        "location": None,
        "time_period": "today",  # Default to today
        "weather_attribute": "general",  # Default to general weather
        "original_question": question
    }

    # Extract location
    # Look for location patterns like "in [location]" or "for [location]"
    location_patterns = [
        r"(?:in|at|for|about)\s+([a-zA-Z\s]+?)(?:\s+(?:tomorrow|today|this week|on|in the|will)|\.|\'s|\?|$)",
        r"([a-zA-Z\s]+?)'s\s+(?:weather|temperature|forecast|rain|precipitation|humidity)",
        r"([a-zA-Z\s]+?)\s+(?:weather|temperature|forecast)"
    ]

    for pattern in location_patterns:
        match = re.search(pattern, question)
        if match:
            location = match.group(1).strip()
            # Filter out common non-location words
            non_locations = ["the", "weather", "it", "there", "here", "outside", "current"]
            if location and location.lower() not in non_locations and len(location) > 2:
                result["location"] = location
                break

    # If no location found, check if question starts with a potential location
    if not result["location"] and not any(question.startswith(w) for w in ["what", "how", "will", "is", "are", "can", "does", "do"]):
        potential_location = question.split()[0]
        if len(potential_location) > 2 and potential_location not in ["tell", "show", "give"]:
            result["location"] = potential_location

    # Extract time period
    time_patterns = {
        "today": [r"today", r"current", r"now", r"right now", r"currently"],
        "tomorrow": [r"tomorrow", r"next day"],
        "day_after_tomorrow": [r"day after tomorrow", r"in two days", r"in 2 days"],
        "this_week": [r"this week", r"coming days", r"next few days", r"forecast"],
        "weekend": [r"weekend", r"saturday", r"sunday"]
    }

    for period, patterns in time_patterns.items():
        if any(re.search(pattern, question) for pattern in patterns):
            result["time_period"] = period
            break

    # Extract weather attribute
    attribute_patterns = {
        "temperature": [r"temperature", r"hot", r"cold", r"warm", r"cool", r"degrees", r"°C", r"°F", r"celsius", r"fahrenheit"],
        "precipitation": [r"rain", r"snow", r"precipitation", r"raining", r"snowing", r"rainfall", r"rainy", r"wet", r"shower"],
        "wind": [r"wind", r"windy", r"breeze", r"breezy", r"gust"],
        "humidity": [r"humid", r"humidity", r"moisture", r"damp"],
        "general": [r"weather", r"forecast", r"condition", r"like outside"]
    }

    for attribute, patterns in attribute_patterns.items():
        if any(re.search(pattern, question) for pattern in patterns):
            result["weather_attribute"] = attribute
            break

    return result

In [11]:
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    # Check for errors in weather data
    if "error" in weather_data:
        return f"Sorry, I couldn't get the weather information: {weather_data['error']}"

    # Extract information from parsed question
    location = parsed_question.get("location")
    time_period = parsed_question.get("time_period", "today")
    attribute = parsed_question.get("weather_attribute", "general")

    # Get location name from weather data
    location_name = weather_data["location"].get("name", "the requested location")
    country_name = weather_data["location"].get("country", "")
    full_location = f"{location_name}, {country_name}" if country_name else location_name

    # Handle current weather
    if time_period == "today":
        current = weather_data["current"]

        if attribute == "temperature":
            temp = current["temp_C"]
            feels_like = current["feels_like_C"]

            if temp == feels_like:
                return f"The current temperature in {full_location} is {temp}°C."
            else:
                return f"The current temperature in {full_location} is {temp}°C, but it feels like {feels_like}°C."

        elif attribute == "precipitation":
            precip = current["precipitation_mm"]
            weather_desc = current["weather_desc"]

            if precip > 0:
                return f"There is currently {precip}mm of precipitation in {full_location}. The conditions are described as {weather_desc}."
            else:
                return f"There is currently no precipitation in {full_location}. The conditions are described as {weather_desc}."

        elif attribute == "wind":
            wind_speed = current["wind_speed_kmph"]
            wind_dir = current["wind_direction"]

            return f"The current wind speed in {full_location} is {wind_speed} km/h from the {wind_dir} direction."

        elif attribute == "humidity":
            humidity = current["humidity"]

            return f"The current humidity in {full_location} is {humidity}%."

        else:  # general weather
            temp = current["temp_C"]
            weather_desc = current["weather_desc"]

            return f"Currently in {full_location}, it's {temp}°C with {weather_desc}."

    # Handle forecast weather
    else:
        # Map time period to forecast index
        forecast_index = 0  # today
        if time_period == "tomorrow":
            forecast_index = 1
        elif time_period == "day_after_tomorrow":
            forecast_index = 2

        # Check if forecast data is available for the requested time period
        if forecast_index >= len(weather_data["forecast"]):
            return f"Sorry, I don't have forecast data for that time period in {full_location}."

        forecast = weather_data["forecast"][forecast_index]
        date_obj = datetime.strptime(forecast["date"], "%Y-%m-%d")
        date_str = date_obj.strftime("%A, %B %d")

        if attribute == "temperature":
            max_temp = forecast["max_temp_C"]
            min_temp = forecast["min_temp_C"]

            if time_period == "tomorrow":
                return f"Tomorrow in {full_location}, expect temperatures between {min_temp}°C and {max_temp}°C."
            else:
                return f"On {date_str} in {full_location}, expect temperatures between {min_temp}°C and {max_temp}°C."

        elif attribute == "precipitation":
            precip = forecast["total_precipitation_mm"]

            if precip > 0:
                if time_period == "tomorrow":
                    return f"Tomorrow in {full_location}, expect around {precip}mm of precipitation."
                else:
                    return f"On {date_str} in {full_location}, expect around {precip}mm of precipitation."
            else:
                if time_period == "tomorrow":
                    return f"No precipitation is expected tomorrow in {full_location}."
                else:
                    return f"No precipitation is expected on {date_str} in {full_location}."

        else:  # general weather or other attributes
            max_temp = forecast["max_temp_C"]
            min_temp = forecast["min_temp_C"]
            precip = forecast["total_precipitation_mm"]

            precip_str = f" with around {precip}mm of precipitation" if precip > 0 else " with no precipitation expected"

            if time_period == "tomorrow":
                return f"Tomorrow in {full_location}, expect temperatures between {min_temp}°C and {max_temp}°C{precip_str}."
            else:
                return f"On {date_str} in {full_location}, expect temperatures between {min_temp}°C and {max_temp}°C{precip_str}."

    # Fallback response
    return f"I have weather information for {full_location}, but I'm not sure how to answer your specific question."

## 🧭 User Interface

In [12]:
def display_menu():
    """
    Display the main menu and handle user input.
    """
    while True:
        print("\n" + "=" * 50)
        print("🌦️  WEATHERWISE - Your Intelligent Weather Advisor  🌦️")
        print("=" * 50)

        options = [
            "Get current weather",
            "View weather forecast",
            "Ask a weather question",
            "View temperature visualization",
            "View precipitation visualization",
            "Exit"
        ]

        choice = pyip.inputMenu(options, numbered=True)

        if choice == "Get current weather":
            get_current_weather()
        elif choice == "View weather forecast":
            view_weather_forecast()
        elif choice == "Ask a weather question":
            ask_weather_question()
        elif choice == "View temperature visualization":
            view_temperature_visualization()
        elif choice == "View precipitation visualization":
            view_precipitation_visualization()
        elif choice == "Exit":
            print("\nThank you for using WeatherWise! Goodbye.")
            break

In [13]:
def get_current_weather():
    """
    Get and display current weather for a location.
    """
    print("\n📍 Get Current Weather")
    location = pyip.inputStr("Enter location (city, country): ", blank=False)

    print(f"\nFetching current weather for {location}...")
    weather_data = get_weather_data(location)

    if "error" in weather_data:
        print(f"\nError: {weather_data['error']}")
        return

    # Display current weather information
    location_name = weather_data["location"].get("name", "Unknown")
    country_name = weather_data["location"].get("country", "")
    full_location = f"{location_name}, {country_name}" if country_name else location_name

    current = weather_data["current"]

    print("\n" + "-" * 50)
    print(f"Current Weather for {full_location}")
    print("-" * 50)
    print(f"Temperature: {current['temp_C']}°C ({current['temp_F']}°F)")
    print(f"Feels Like: {current['feels_like_C']}°C")
    print(f"Conditions: {current['weather_desc']}")
    print(f"Humidity: {current['humidity']}%")
    print(f"Wind: {current['wind_speed_kmph']} km/h, {current['wind_direction']}")
    print(f"Precipitation: {current['precipitation_mm']} mm")
    print(f"Visibility: {current['visibility_km']} km")
    print(f"UV Index: {current['uv_index']}")
    print(f"Observation Time: {current['observation_time']}")
    print("-" * 50)

    # Ask if user wants to see visualizations
    show_viz = pyip.inputYesNo("\nWould you like to see weather visualizations? (yes/no): ")
    if show_viz == "yes":
        create_temperature_visualization(weather_data)
        create_precipitation_visualization(weather_data)

In [14]:
def view_weather_forecast():
    """
    View weather forecast for a location.
    """
    print("\n🔮 View Weather Forecast")
    location = pyip.inputStr("Enter location (city, country): ", blank=False)
    days = pyip.inputInt("Enter number of days (1-3): ", min=1, max=3)

    print(f"\nFetching {days}-day forecast for {location}...")
    weather_data = get_weather_data(location, days)

    if "error" in weather_data:
        print(f"\nError: {weather_data['error']}")
        return

    # Display forecast information
    location_name = weather_data["location"].get("name", "Unknown")
    country_name = weather_data["location"].get("country", "")
    full_location = f"{location_name}, {country_name}" if country_name else location_name

    print("\n" + "-" * 50)
    print(f"{days}-Day Weather Forecast for {full_location}")
    print("-" * 50)

    for day in weather_data["forecast"]:
        date_obj = datetime.strptime(day["date"], "%Y-%m-%d")
        date_str = date_obj.strftime("%A, %B %d")

        print(f"\n📅 {date_str}")
        print(f"Temperature Range: {day['min_temp_C']}°C to {day['max_temp_C']}°C")
        print(f"Total Precipitation: {day['total_precipitation_mm']} mm")
        print(f"Sunrise: {day['sunrise']}")
        print(f"Sunset: {day['sunset']}")

        # Ask if user wants to see hourly details
        show_hourly = pyip.inputYesNo(f"\nShow hourly details for {date_str}? (yes/no): ")
        if show_hourly == "yes":
            print("\nHourly Forecast:")
            print("-" * 30)
            for hour in day["hourly"]:
                print(f"Time: {hour['time']}")
                print(f"Temperature: {hour['temp_C']}°C (Feels like: {hour['feels_like_C']}°C)")
                print(f"Conditions: {hour['weather_desc']}")
                print(f"Precipitation: {hour['precipitation_mm']} mm (Chance of rain: {hour['chance_of_rain']}%)")
                print(f"Wind: {hour['wind_speed_kmph']} km/h, {hour['wind_direction']}")
                print("-" * 30)

    # Ask if user wants to see visualizations
    show_viz = pyip.inputYesNo("\nWould you like to see weather visualizations? (yes/no): ")
    if show_viz == "yes":
        create_temperature_visualization(weather_data)
        create_precipitation_visualization(weather_data)

In [15]:
def ask_weather_question():
    """
    Ask a natural language weather question.
    """
    print("\n❓ Ask a Weather Question")
    print("Examples:")
    print("- What's the weather like in London today?")
    print("- Will it rain in New York tomorrow?")
    print("- How hot will it be in Tokyo this week?")
    print("- What's the temperature in Paris?")

    question = pyip.inputStr("\nEnter your weather question: ", blank=False)

    # Parse the question
    parsed = parse_weather_question(question)

    # If no location was found, ask for it
    if not parsed["location"]:
        location = pyip.inputStr("I couldn't determine the location. Please specify a location: ", blank=False)
        parsed["location"] = location

    print(f"\nFetching weather data for {parsed['location']}...")
    weather_data = get_weather_data(parsed["location"])

    if "error" in weather_data:
        print(f"\nError: {weather_data['error']}")
        return

    # Generate and display response
    response = generate_weather_response(parsed, weather_data)

    print("\n" + "-" * 50)
    print("🤖 WeatherWise Response:")
    print("-" * 50)
    print(response)
    print("-" * 50)

    # Show what was understood from the question
    print("\n📝 I understood your question as:")
    print(f"Location: {parsed['location']}")
    print(f"Time Period: {parsed['time_period']}")
    print(f"Weather Attribute: {parsed['weather_attribute']}")

    # Ask if user wants to see visualizations
    show_viz = pyip.inputYesNo("\nWould you like to see weather visualizations? (yes/no): ")
    if show_viz == "yes":
        create_temperature_visualization(weather_data)
        create_precipitation_visualization(weather_data)

In [16]:
def view_temperature_visualization():
    """
    View temperature visualization for a location.
    """
    print("\n🌡️ View Temperature Visualization")
    location = pyip.inputStr("Enter location (city, country): ", blank=False)
    days = pyip.inputInt("Enter number of days (1-3): ", min=1, max=3)

    print(f"\nFetching temperature data for {location}...")
    weather_data = get_weather_data(location, days)

    if "error" in weather_data:
        print(f"\nError: {weather_data['error']}")
        return

    # Create and display visualization
    create_temperature_visualization(weather_data)

In [17]:
def view_precipitation_visualization():
    """
    View precipitation visualization for a location.
    """
    print("\n🌧️ View Precipitation Visualization")
    location = pyip.inputStr("Enter location (city, country): ", blank=False)
    days = pyip.inputInt("Enter number of days (1-3): ", min=1, max=3)

    print(f"\nFetching precipitation data for {location}...")
    weather_data = get_weather_data(location, days)

    if "error" in weather_data:
        print(f"\nError: {weather_data['error']}")
        return

    # Create and display visualization
    create_precipitation_visualization(weather_data)

## 🧩 Main Application Logic

In [18]:
# Run the WeatherWise application
if __name__ == "__main__":
    display_menu()


🌦️  WEATHERWISE - Your Intelligent Weather Advisor  🌦️
Please select one of the following:
1. Get current weather
2. View weather forecast
3. Ask a weather question
4. View temperature visualization
5. View precipitation visualization
6. Exit
6

Thank you for using WeatherWise! Goodbye.


## 🧪 Testing and Examples

In [19]:
# Test weather data retrieval
print("Testing weather data retrieval for London...")
london_weather = get_weather_data("London")

# Check if the weather_data dictionary contains an error key
if "error" in london_weather:
    print(f"Error retrieving weather data: {london_weather['error']}")
else:
    print(f"Location: {london_weather['location']['name']}, {london_weather['location']['country']}")
    print(f"Current temperature: {london_weather['current']['temp_C']}°C")
    print(f"Weather description: {london_weather['current']['weather_desc']}")
    print("Weather data retrieval test completed.\n")

# Test natural language processing
test_questions = [
    "What's the weather like in Paris today?",
    "Will it rain in New York tomorrow?",
    "How hot will it be in Tokyo this week?",
    "What's the temperature in London?",
    "Is it windy in Chicago?"
]

print("Testing natural language processing...")
for question in test_questions:
    parsed = parse_weather_question(question)
    print(f"\nQuestion: {question}")
    print(f"Parsed: Location={parsed['location']}, Time={parsed['time_period']}, Attribute={parsed['weather_attribute']}")
print("Natural language processing test completed.\n")

# Test visualizations
print("Testing temperature visualization for London...")
# Only attempt to visualize if weather data was successfully retrieved
if "error" not in london_weather:
    create_temperature_visualization(london_weather)
else:
    print("Skipping temperature visualization test due to retrieval error.")

print("\nTesting precipitation visualization for London...")
# Only attempt to visualize if weather data was successfully retrieved
if "error" not in london_weather:
    create_precipitation_visualization(london_weather)
else:
    print("Skipping precipitation visualization test due to retrieval error.")

print("Visualization tests completed.")

Testing weather data retrieval for London...
Unexpected error: Expecting value: line 1 column 1 (char 0)
Error retrieving weather data: Unexpected error: Expecting value: line 1 column 1 (char 0)
Testing natural language processing...

Question: What's the weather like in Paris today?
Parsed: Location=paris, Time=today, Attribute=general

Question: Will it rain in New York tomorrow?
Parsed: Location=in new york, Time=tomorrow, Attribute=precipitation

Question: How hot will it be in Tokyo this week?
Parsed: Location=tokyo, Time=this_week, Attribute=temperature

Question: What's the temperature in London?
Parsed: Location=london, Time=today, Attribute=temperature

Question: Is it windy in Chicago?
Parsed: Location=chicago, Time=today, Attribute=wind
Natural language processing test completed.

Testing temperature visualization for London...
Skipping temperature visualization test due to retrieval error.

Testing precipitation visualization for London...
Skipping precipitation visualizat

In [20]:
# Test natural language processing
test_questions = [
    "What's the weather like in Paris today?",
    "Will it rain in New York tomorrow?",
    "How hot will it be in Tokyo this week?",
    "What's the temperature in London?",
    "Is it windy in Chicago?"
]

print("Testing natural language processing...")
for question in test_questions:
    parsed = parse_weather_question(question)
    print(f"\nQuestion: {question}")
    print(f"Parsed: Location={parsed['location']}, Time={parsed['time_period']}, Attribute={parsed['weather_attribute']}")
print("Natural language processing test completed.\n")

Testing natural language processing...

Question: What's the weather like in Paris today?
Parsed: Location=paris, Time=today, Attribute=general

Question: Will it rain in New York tomorrow?
Parsed: Location=in new york, Time=tomorrow, Attribute=precipitation

Question: How hot will it be in Tokyo this week?
Parsed: Location=tokyo, Time=this_week, Attribute=temperature

Question: What's the temperature in London?
Parsed: Location=london, Time=today, Attribute=temperature

Question: Is it windy in Chicago?
Parsed: Location=chicago, Time=today, Attribute=wind
Natural language processing test completed.



In [21]:
# Test visualizations
print("Testing temperature visualization for London...")
create_temperature_visualization(london_weather)
print("\nTesting precipitation visualization for London...")
create_precipitation_visualization(london_weather)
print("Visualization tests completed.")

Testing temperature visualization for London...
Error: Unexpected error: Expecting value: line 1 column 1 (char 0)

Testing precipitation visualization for London...
Error: Unexpected error: Expecting value: line 1 column 1 (char 0)
Visualization tests completed.


[20]
1s
# Test weather data retrieval
print("Testing weather data retrieval for London...")
london_weather = get_weather_data("London")

# Check if the weather_data dictionary contains an error key
if "error" in london_weather:
    print(f"Error retrieving weather data: {london_weather['error']}")
else:
    print(f"Location: {london_weather['location']['name']}, {london_weather['location']['country']}")
    print(f"Current temperature: {london_weather['current']['temp_C']}°C")
…    create_precipitation_visualization(london_weather)
else:
    print("Skipping precipitation visualization test due to retrieval error.")

print("Visualization tests completed.")
Testing weather data retrieval for London...
Unexpected error: Expecting value: line 1 column 1 (char 0)
Error retrieving weather data: Unexpected error: Expecting value: line 1 column 1 (char 0)
Testing natural language processing...

Question: What's the weather like in Paris today?
Parsed: Location=paris, Time=today, Attribute=general

Question: Will it rain in New York tomorrow?
Parsed: Location=in new york, Time=tomorrow, Attribute=precipitation

Question: How hot will it be in Tokyo this week?
Parsed: Location=tokyo, Time=this_week, Attribute=temperature

Question: What's the temperature in London?
Parsed: Location=london, Time=today, Attribute=temperature

Question: Is it windy in Chicago?
Parsed: Location=chicago, Time=today, Attribute=wind
Natural language processing test completed.

Testing temperature visualization for London...
Skipping temperature visualization test due to retrieval error.

Testing precipitation visualization for London...
Skipping precipitation visualization test due to retrieval error.
Visualization tests completed.

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.

# 🗂️ AI Prompting Log

This section documents the AI-assisted development process for the WeatherWise application.

## AI Conversations

Five significant AI conversations were conducted during the development of this application:

1. **Implementation Options Exploration** - Discussed different weather data sources and their pros/cons
2. **Natural Language Processing Design** - Explored approaches for parsing weather questions
3. **Visualization Techniques** - Discussed effective ways to visualize temperature and precipitation data
4. **Error Handling Strategies** - Developed robust error handling for API requests and user inputs
5. **User Interface Design** - Created a user-friendly console interface with pyinputplus

## Intentional Prompting Techniques

Several intentional prompting techniques were used during development:

1. **Specific Task Definition** - Clearly defined the task and expected output
2. **Iterative Refinement** - Started with basic implementations and refined through feedback
3. **Code Review Requests** - Asked for specific improvements to initial code
4. **Error Scenario Exploration** - Prompted for handling of specific error cases
5. **Before/After Comparisons** - Compared initial and improved implementations

## Before/After Examples

### Example 1: Weather Data Retrieval

**Before:**
```python
def get_weather_data(location):
    url = f"https://wttr.in/{location}?format=j1"
    response = requests.get(url)
    return response.json()
```

**After:**
```python
def get_weather_data(location, forecast_days=3):
    # Input validation
    if not location or not isinstance(location, str):
        location = DEFAULT_LOCATION
        print(f"Invalid location. Using default: {DEFAULT_LOCATION}")
    
    # Check cache first
    cache_file = f"cache/{location.lower().replace(' ', '_')}.json"
    if os.path.exists(cache_file):
        with open(cache_file, 'r') as f:
            cache_data = json.load(f)
            # Check if cache is still valid
            if time.time() - cache_data.get('timestamp', 0) < CACHE_DURATION:
                print(f"Using cached data for {location}")
                return cache_data['data']
    
    # Prepare API request with error handling
    url = f"{WEATHER_API_BASE_URL}{location}?format={WEATHER_API_FORMAT}&lang=en"
    
    try:
        # Make API request with timeout
        response = requests.get(url, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()
        # Process and structure the data
        # Save to cache
        # Return processed data
    except (requests.exceptions.Timeout, requests.exceptions.ConnectionError,
            requests.exceptions.HTTPError, json.JSONDecodeError) as e:
        # Handle specific errors
        return {"error": str(e)}
```

### Example 2: Natural Language Processing

**Before:**
```python
def parse_weather_question(question):
    words = question.lower().split()
    location = None
    time_period = "today"
    
    if "tomorrow" in words:
        time_period = "tomorrow"
    
    for word in words:
        if word not in ["what", "is", "the", "weather", "like", "in", "tomorrow", "today"]:
            location = word
            break
    
    return {"location": location, "time_period": time_period}
```

**After:**
```python
def parse_weather_question(question):
    # Convert question to lowercase
    question = question.lower()
    
    # Initialize result dictionary
    result = {
        "location": None,
        "time_period": "today",
        "weather_attribute": "general",
        "original_question": question
    }
    
    # Extract location using regex patterns
    location_patterns = [
        r"(?:in|at|for|about)\s+([a-zA-Z\s]+?)(?:\s+(?:tomorrow|today|this week|on|in the|will)|\.|\'s|\?|$)",
        r"([a-zA-Z\s]+?)'s\s+(?:weather|temperature|forecast|rain|precipitation|humidity)",
        r"([a-zA-Z\s]+?)\s+(?:weather|temperature|forecast)"
    ]
    
    # Extract time period using pattern matching
    time_patterns = {
        "today": [r"today", r"current", r"now"],
        "tomorrow": [r"tomorrow", r"next day"],
        "this_week": [r"this week", r"coming days", r"forecast"]
    }
    
    # Extract weather attribute using pattern matching
    attribute_patterns = {
        "temperature": [r"temperature", r"hot", r"cold"],
        "precipitation": [r"rain", r"snow", r"precipitation"],
        "wind": [r"wind", r"windy", r"breeze"],
        "general": [r"weather", r"forecast", r"condition"]
    }
    
    # Apply pattern matching and return structured result
```

### Example 3: Visualization

**Before:**
```python
def create_temperature_visualization(weather_data):
    dates = [day["date"] for day in weather_data["forecast"]]
    temps = [day["max_temp_C"] for day in weather_data["forecast"]]
    
    plt.figure(figsize=(10, 5))
    plt.plot(dates, temps)
    plt.title("Temperature Forecast")
    plt.xlabel("Date")
    plt.ylabel("Temperature (°C)")
    plt.show()
```

**After:**
```python
def create_temperature_visualization(weather_data, output_type='display'):
    # Error handling
    if "error" in weather_data:
        print(f"Error: {weather_data['error']}")
        return None
    
    # Create figure and axis with proper sizing
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Prepare data for plotting
    dates = []
    max_temps = []
    min_temps = []
    
    # Extract and format data
    for day in weather_data["forecast"]:
        date_obj = datetime.strptime(day["date"], "%Y-%m-%d")
        dates.append(date_obj)
        max_temps.append(day["max_temp_C"])
        min_temps.append(day["min_temp_C"])
    
    # Plot with enhanced styling
    ax.plot(dates, max_temps, 'o-', color=TEMP_COLORS["high"], linewidth=2,
            markersize=8, label='Max Temperature')
    ax.plot(dates, min_temps, 'o-', color=TEMP_COLORS["low"], linewidth=2,
            markersize=8, label='Min Temperature')
    
    # Add current temperature reference
    current_temp = weather_data["current"]["temp_C"]
    ax.axhline(y=current_temp, color='green', linestyle='--',
               alpha=0.7, label='Current Temperature')
    
    # Add data labels, proper formatting, and styling
    # Return or display based on output_type parameter
```